In [1]:
%%writefile DBSCAN_100_GPU.cu

#include <iostream>
#include <fstream>
#include <sstream>
#include <cmath>
#include <stack>
#include <cuda_runtime.h>

using namespace std;

// Параметры:
const int size_mass = 100;
const int dim = 3;
const float epsilon = 1.43f;
const int minPts = 3;


// Объявление константной памяти на GPU
__constant__ int d_size_mass;
__constant__ int d_dim;
__constant__ float d_epsilon;
__constant__ int d_minPts;


// Путь к файлу с данными
string file = "my_mass100.txt";

// Функция для считывания из файла данных с размерностью (100, 3)
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}


// ядро для обнуления
__global__ void nullCountCUDA(int* count) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        *count = 0;
    }
}


// функция для поиска соседей у точки, результат в виде 0 и 1 будет лежать в массиве neighbours
__global__ void findNeighboursCUDA(float* data, int point_index, int* neighbours, int* count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < d_size_mass) {
        if (idx != point_index) {
            // вычисление евклидового расстояния
            float sum = 0;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += (data[point_index * d_dim + coord] - data[idx * d_dim + coord]) * (data[point_index * d_dim + coord] - data[idx * d_dim + coord]);
            }
            float distance = sqrt(sum);
            // проверка, входит ли точка в окрестность
            if (distance <= d_epsilon) {
                neighbours[idx] = 1;
            } else {
                neighbours[idx] = 0;
            }
        } else {
            neighbours[idx] = 0;
        }
        // для подсчёта соседей
        if (neighbours[idx] == 1) {
            atomicAdd(count, 1);
        }
    }
}

// функция для расширения кластера
void expand_cluster(float* data, int start_point, int cluster_id, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // создание стека для отслеживания точек, которые нужно обработать
    stack<int> stack;

    // В стек добавляется начальная точка
    stack.push(start_point);

    // Пока стек не пустой
    while (!stack.empty()) {
        // получение точки из стека
        int point = stack.top();
        // удаление точки из стека
        stack.pop();

        // назначение точке метки текущего кластера
        labels[point] = cluster_id;

        // Запуск ядра для обнуление счётчика соседей
        nullCountCUDA<<<1, 1>>>(d_count);
        cudaDeviceSynchronize();

        // Поиск соседей
        findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point, d_neighbours, d_count);
        cudaDeviceSynchronize();

        int count;
        // копирование найденного количества соседей с GPU на CPU
        cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

        if (count >= minPts) {
            int* neighbours = new int[size_mass];
            cudaMemcpy(neighbours, d_neighbours, size_mass * sizeof(int), cudaMemcpyDeviceToHost);
            for (int i = 0; i < size_mass; i++) {
                if (neighbours[i] == 1) {
                    if (labels[i] == -1) {
                        stack.push(i);
                        labels[i] = cluster_id;
                    }
                    if (labels[i] == -2) {
                        labels[i] = cluster_id;
                    }
                }
            }
            delete[] neighbours;
        }
    }
}

// Основная фнункция алгоритма
int dbscan(float* data, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_epsilon, &epsilon, sizeof(float));
    cudaMemcpyToSymbol(d_minPts, &minPts, sizeof(int));

    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // Начальное значение индекса кластера
    int cluster_id = 0;

    for (int point_index = 0; point_index < size_mass; point_index++) {
        // если точка не посещена
        if (labels[point_index] == -1) {
            // Запуск ядра для обнуление счётчика соседей
            nullCountCUDA<<<1, 1>>>(d_count);
            cudaDeviceSynchronize();

            // Поиск соседей
            findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point_index, d_neighbours, d_count);
            cudaDeviceSynchronize();

            int count;
            // копирование найденного количества соседей с GPU на CPU
            cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

            // Если количество соседей меньше заданного минимума
            if (count < minPts) {
                // Точка помечается как шумовая (-2)
                labels[point_index] = -2;
                continue;
            }

            // Если больше или равно
            if (count >= minPts) {
                // Расширение кластера
                expand_cluster(data, point_index, cluster_id, labels, d_data, d_neighbours, d_count);
                // После того, как кластер завершён, увеличивается индекс для текущего
                cluster_id++;
            }
        }
    }

    return cluster_id;
}

int main() {

    // Выделение памяти

    // Данные с точками
    float* myArray = new float[size_mass * dim];

    // для меток точек, изначально не посещённые (-1)
    int* labels = new int[size_mass];
    for (int i = 0; i < size_mass; i++) {
        labels[i] = -1;
    }

    // чтение данных их файла и запись в массив
    readFromFile(myArray, file);

    // Указатели на GPU память
    float* dev_data;
    int* dev_neighbours;
    int* d_count;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_neighbours, size_mass * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));

    // Копирование данных на GPU
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // работа алгоритма DBSCAN, вернёт количество кластеров
    int count_clusters = dbscan(myArray, labels, dev_data, dev_neighbours, d_count);

    // Вывод меток
    cout << endl << "Labels: " << endl;
    for (int i = 0; i < size_mass; i++) {
        cout << labels[i] << " ";
    }

    // Группировка меток по кластерам
    cout << endl << "Clusters." << endl;
    for (int cluster_id = 0; cluster_id < count_clusters; cluster_id++) {
        cout << cluster_id << ": ";
        for (int point_index = 0; point_index < size_mass; point_index++) {
            if (labels[point_index] == cluster_id) {
                cout << point_index << " ";
            }
        }
        cout << endl;
    }

    cout << "Noise: ";
    for (int point_index = 0; point_index < size_mass; point_index++) {
        if (labels[point_index] == -2) {
            cout << point_index << " ";
        }
    }
    cout << endl << endl;

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_neighbours);
    cudaFree(d_count);

    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] labels;

    return 0;
}

Writing DBSCAN_100_GPU.cu


In [2]:
!nvcc DBSCAN_100_GPU.cu -o DBSCAN_100_GPU
# компилляция

In [3]:
# Запуск
!./DBSCAN_100_GPU


Labels: 
0 0 0 0 -2 0 0 0 0 0 -2 0 -2 0 -2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 -2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 -2 2 2 2 2 2 2 2 2 2 2 2 2 -2 2 2 2 2 2 2 -2 2 
Clusters.
0: 0 1 2 3 5 6 7 8 9 11 13 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 73 
1: 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
2: 60 61 62 63 64 65 66 67 68 69 70 71 72 74 75 76 77 79 80 81 82 83 84 85 86 87 88 89 90 92 93 94 95 96 97 99 
Noise: 4 10 12 14 30 45 78 91 98 



In [ ]:
%%writefile DBSCANTIME_100_GPU.cu


#include <iostream>
#include <fstream>
#include <sstream>
#include <cmath>
#include <stack>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 10;
float totalMallocTime = 0.0f;
float totalKmTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;



// Параметры:
const int size_mass = 100;
const int dim = 3;
const float epsilon = 1.43f;
const int minPts = 3;


// Объявление константной памяти на GPU
__constant__ int d_size_mass;
__constant__ int d_dim;
__constant__ float d_epsilon;
__constant__ int d_minPts;


// Путь к файлу с данными
string file = "my_mass100.txt";

// Функция для считывания из файла данных с размерностью (100, 3)
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}


// ядро для обнуления
__global__ void nullCountCUDA(int* count) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        *count = 0;
    }
}


// функция для поиска соседей у точки, результат в виде 0 и 1 будет лежать в массиве neighbours
__global__ void findNeighboursCUDA(float* data, int point_index, int* neighbours, int* count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < d_size_mass) {
        if (idx != point_index) {
            // вычисление евклидового расстояния
            float sum = 0;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += (data[point_index * d_dim + coord] - data[idx * d_dim + coord]) * (data[point_index * d_dim + coord] - data[idx * d_dim + coord]);
            }
            float distance = sqrt(sum);
            // проверка, входит ли точка в окрестность
            if (distance <= d_epsilon) {
                neighbours[idx] = 1;
            } else {
                neighbours[idx] = 0;
            }
        } else {
            neighbours[idx] = 0;
        }
        // для подсчёта соседей
        if (neighbours[idx] == 1) {
            atomicAdd(count, 1);
        }
    }
}

// функция для расширения кластера
void expand_cluster(float* data, int start_point, int cluster_id, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // создание стека для отслеживания точек, которые нужно обработать
    stack<int> stack;

    // В стек добавляется начальная точка
    stack.push(start_point);

    // Пока стек не пустой
    while (!stack.empty()) {
        // получение точки из стека
        int point = stack.top();
        // удаление точки из стека
        stack.pop();

        // назначение точке метки текущего кластера
        labels[point] = cluster_id;

        // Запуск ядра для обнуление счётчика соседей
        nullCountCUDA<<<1, 1>>>(d_count);
        cudaDeviceSynchronize();

        // Поиск соседей
        findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point, d_neighbours, d_count);
        cudaDeviceSynchronize();

        int count;
        // копирование найденного количества соседей с GPU на CPU
        cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

        if (count >= minPts) {
            int* neighbours = new int[size_mass];
            cudaMemcpy(neighbours, d_neighbours, size_mass * sizeof(int), cudaMemcpyDeviceToHost);
            for (int i = 0; i < size_mass; i++) {
                if (neighbours[i] == 1) {
                    if (labels[i] == -1) {
                        stack.push(i);
                        labels[i] = cluster_id;
                    }
                    if (labels[i] == -2) {
                        labels[i] = cluster_id;
                    }
                }
            }
            delete[] neighbours;
        }
    }
}

// Основная фнункция алгоритма
int dbscan(float* data, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_epsilon, &epsilon, sizeof(float));
    cudaMemcpyToSymbol(d_minPts, &minPts, sizeof(int));

    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // Начальное значение индекса кластера
    int cluster_id = 0;

    for (int point_index = 0; point_index < size_mass; point_index++) {
        // если точка не посещена
        if (labels[point_index] == -1) {
            // Запуск ядра для обнуление счётчика соседей
            nullCountCUDA<<<1, 1>>>(d_count);
            cudaDeviceSynchronize();

            // Поиск соседей
            findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point_index, d_neighbours, d_count);
            cudaDeviceSynchronize();

            int count;
            // копирование найденного количества соседей с GPU на CPU
            cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

            // Если количество соседей меньше заданного минимума
            if (count < minPts) {
                // Точка помечается как шумовая (-2)
                labels[point_index] = -2;
                continue;
            }

            // Если больше или равно
            if (count >= minPts) {
                // Расширение кластера
                expand_cluster(data, point_index, cluster_id, labels, d_data, d_neighbours, d_count);
                // После того, как кластер завершён, увеличивается индекс для текущего
                cluster_id++;
            }
        }
    }

    return cluster_id;
}

int main() {


//  ***************** Начало замеров **********************
    for (int iter = 0; iter < iterations; iter++) {

    // Выделение памяти

    // Данные с точками
    float* myArray = new float[size_mass * dim];

    // для меток точек, изначально не посещённые (-1)
    int* labels = new int[size_mass];
    for (int i = 0; i < size_mass; i++) {
        labels[i] = -1;
    }

    // чтение данных их файла и запись в массив
    readFromFile(myArray, file);


        // -------------------------------------------------------------------------
        // замеры времени выделения памяти на GPU с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

    // Указатели на GPU память
    float* dev_data;
    int* dev_neighbours;
    int* d_count;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_neighbours, size_mass * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));


        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------

        // __________________
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);


    // Копирование данных на GPU
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);



        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // __________________



        // ...........................................................................
        // замеры времени работы алгоритма DBSCAN с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);

    // работа алгоритма DBSCAN, вернёт количество кластеров
    int count_clusters = dbscan(myArray, labels, dev_data, dev_neighbours, d_count);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float kmTime;
        cudaEventElapsedTime(&kmTime, start3, stop3);
        totalKmTime += kmTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................


    if (iter == 0){
    // Вывод меток
    cout << endl << "Labels: " << endl;
    for (int i = 0; i < size_mass; i++) {
        cout << labels[i] << " ";
    }

    // Группировка меток по кластерам
    cout << endl << "Clusters." << endl;
    for (int cluster_id = 0; cluster_id < count_clusters; cluster_id++) {
        cout << cluster_id << ": ";
        for (int point_index = 0; point_index < size_mass; point_index++) {
            if (labels[point_index] == cluster_id) {
                cout << point_index << " ";
            }
        }
        cout << endl;
    }

    cout << "Noise: ";
    for (int point_index = 0; point_index < size_mass; point_index++) {
        if (labels[point_index] == -2) {
            cout << point_index << " ";
        }
    }
    cout << endl << endl;
}

        // .............................................................
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_neighbours);
    cudaFree(d_count);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ..................................................................

    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] labels;


}
// *********************** Конец замера времени ***************************



    cout << "\nDBSCAN. For mass 100 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "K-means: " << totalKmTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalKmTime + totalFreeTime) / iterations << " ms" << endl;


    return 0;
}



Overwriting DBSCANTIME_100_GPU.cu


In [ ]:
!nvcc DBSCANTIME_100_GPU.cu -o DBSCANTIME_100_GPU
# компилляция

In [ ]:
# Запуск
!./DBSCANTIME_100_GPU


Labels: 
0 0 0 0 -2 0 0 0 0 0 -2 0 -2 0 -2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 -2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 -2 2 2 2 2 2 2 2 2 2 2 2 2 -2 2 2 2 2 2 2 -2 2 
Clusters.
0: 0 1 2 3 5 6 7 8 9 11 13 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 73 
1: 31 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
2: 60 61 62 63 64 65 66 67 68 69 70 71 72 74 75 76 77 79 80 81 82 83 84 85 86 87 88 89 90 92 93 94 95 96 97 99 
Noise: 4 10 12 14 30 45 78 91 98 


DBSCAN. For mass 100 points
GPU memory allocation time: 0.130144 ms
Time to copy data from the host to the device: 0.0931392 ms
K-means: 16.8798 ms
Free GPU: 0.112067 ms
Total: 17.2151 ms


In [4]:
%%writefile DBSCANTIME_10000_GPU.cu


#include <iostream>
#include <fstream>
#include <sstream>
#include <cmath>
#include <stack>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 10;
float totalMallocTime = 0.0f;
float totalDBSCANTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;



// Параметры:
const int size_mass = 10000;
const int dim = 3;
const float epsilon = 1.43f;
const int minPts = 3;


// Объявление константной памяти на GPU
__constant__ int d_size_mass;
__constant__ int d_dim;
__constant__ float d_epsilon;
__constant__ int d_minPts;


// Путь к файлу с данными
string file = "my_mass10000.txt";

// Функция для считывания из файла данных с размерностью (10000, 3)
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}


// ядро для обнуления
__global__ void nullCountCUDA(int* count) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        *count = 0;
    }
}


// функция для поиска соседей у точки, результат в виде 0 и 1 будет лежать в массиве neighbours
__global__ void findNeighboursCUDA(float* data, int point_index, int* neighbours, int* count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < d_size_mass) {
        if (idx != point_index) {
            // вычисление евклидового расстояния
            float sum = 0;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += (data[point_index * d_dim + coord] - data[idx * d_dim + coord]) * (data[point_index * d_dim + coord] - data[idx * d_dim + coord]);
            }
            float distance = sqrt(sum);
            // проверка, входит ли точка в окрестность
            if (distance <= d_epsilon) {
                neighbours[idx] = 1;
            } else {
                neighbours[idx] = 0;
            }
        } else {
            neighbours[idx] = 0;
        }
        // для подсчёта соседей
        if (neighbours[idx] == 1) {
            atomicAdd(count, 1);
        }
    }
}

// функция для расширения кластера
void expand_cluster(float* data, int start_point, int cluster_id, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // создание стека для отслеживания точек, которые нужно обработать
    stack<int> stack;

    // В стек добавляется начальная точка
    stack.push(start_point);

    // Пока стек не пустой
    while (!stack.empty()) {
        // получение точки из стека
        int point = stack.top();
        // удаление точки из стека
        stack.pop();

        // назначение точке метки текущего кластера
        labels[point] = cluster_id;

        // Запуск ядра для обнуление счётчика соседей
        nullCountCUDA<<<1, 1>>>(d_count);
        cudaDeviceSynchronize();

        // Поиск соседей
        findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point, d_neighbours, d_count);
        cudaDeviceSynchronize();

        int count;
        // копирование найденного количества соседей с GPU на CPU
        cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

        if (count >= minPts) {
            int* neighbours = new int[size_mass];
            cudaMemcpy(neighbours, d_neighbours, size_mass * sizeof(int), cudaMemcpyDeviceToHost);
            for (int i = 0; i < size_mass; i++) {
                if (neighbours[i] == 1) {
                    if (labels[i] == -1) {
                        stack.push(i);
                        labels[i] = cluster_id;
                    }
                    if (labels[i] == -2) {
                        labels[i] = cluster_id;
                    }
                }
            }
            delete[] neighbours;
        }
    }
}

// Основная фнункция алгоритма
int dbscan(float* data, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_epsilon, &epsilon, sizeof(float));
    cudaMemcpyToSymbol(d_minPts, &minPts, sizeof(int));

    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // Начальное значение индекса кластера
    int cluster_id = 0;

    for (int point_index = 0; point_index < size_mass; point_index++) {
        // если точка не посещена
        if (labels[point_index] == -1) {
            // Запуск ядра для обнуление счётчика соседей
            nullCountCUDA<<<1, 1>>>(d_count);
            cudaDeviceSynchronize();

            // Поиск соседей
            findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point_index, d_neighbours, d_count);
            cudaDeviceSynchronize();

            int count;
            // копирование найденного количества соседей с GPU на CPU
            cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

            // Если количество соседей меньше заданного минимума
            if (count < minPts) {
                // Точка помечается как шумовая (-2)
                labels[point_index] = -2;
                continue;
            }

            // Если больше или равно
            if (count >= minPts) {
                // Расширение кластера
                expand_cluster(data, point_index, cluster_id, labels, d_data, d_neighbours, d_count);
                // После того, как кластер завершён, увеличивается индекс для текущего
                cluster_id++;
            }
        }
    }

    return cluster_id;
}

int main() {


//  ***************** Начало замеров **********************
    for (int iter = 0; iter < iterations; iter++) {

    // Выделение памяти

    // Данные с точками
    float* myArray = new float[size_mass * dim];

    // для меток точек, изначально не посещённые (-1)
    int* labels = new int[size_mass];
    for (int i = 0; i < size_mass; i++) {
        labels[i] = -1;
    }

    // чтение данных их файла и запись в массив
    readFromFile(myArray, file);


        // -------------------------------------------------------------------------
        // замеры времени выделения памяти на GPU с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

    // Указатели на GPU память
    float* dev_data;
    int* dev_neighbours;
    int* d_count;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_neighbours, size_mass * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));


        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------

        // __________________
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);


    // Копирование данных на GPU
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);



        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // __________________



        // ...........................................................................
        // замеры времени работы алгоритма DBSCAN с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);

    // работа алгоритма DBSCAN, вернёт количество кластеров
    int count_clusters = dbscan(myArray, labels, dev_data, dev_neighbours, d_count);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float DBSCANTime;
        cudaEventElapsedTime(&DBSCANTime, start3, stop3);
        totalDBSCANTime += DBSCANTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................


        // .............................................................
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_neighbours);
    cudaFree(d_count);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ..................................................................

    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] labels;


}
// *********************** Конец замера времени ***************************



    cout << "\nDBSCAN. For mass 10000 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "DBSCAN: " << totalDBSCANTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalDBSCANTime + totalFreeTime) / iterations << " ms" << endl;


    return 0;
}



Writing DBSCANTIME_10000_GPU.cu


In [5]:
!nvcc DBSCANTIME_10000_GPU.cu -o DBSCANTIME_10000_GPU
# компилляция

In [8]:
# Запуск
!./DBSCANTIME_10000_GPU


DBSCAN. For mass 10000 points
GPU memory allocation time: 0.20631 ms
Time to copy data from the host to the device: 0.0581888 ms
DBSCAN: 822.607 ms
Free GPU: 0.213699 ms
Total: 823.085 ms


In [ ]:
%%writefile DBSCANTIME_100000_GPU.cu


#include <iostream>
#include <fstream>
#include <sstream>
#include <cmath>
#include <stack>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 1;
float totalMallocTime = 0.0f;
float totalDBSCANTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;



// Параметры:
const int size_mass = 100000;
const int dim = 3;
const float epsilon = 1.43f;
const int minPts = 3;


// Объявление константной памяти на GPU
__constant__ int d_size_mass;
__constant__ int d_dim;
__constant__ float d_epsilon;
__constant__ int d_minPts;


// Путь к файлу с данными
string file = "my_mass100000.txt";

// Функция для считывания из файла данных с размерностью (100000, 3)
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}


// ядро для обнуления
__global__ void nullCountCUDA(int* count) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        *count = 0;
    }
}


// функция для поиска соседей у точки, результат в виде 0 и 1 будет лежать в массиве neighbours
__global__ void findNeighboursCUDA(float* data, int point_index, int* neighbours, int* count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < d_size_mass) {
        if (idx != point_index) {
            // вычисление евклидового расстояния
            float sum = 0;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += (data[point_index * d_dim + coord] - data[idx * d_dim + coord]) * (data[point_index * d_dim + coord] - data[idx * d_dim + coord]);
            }
            float distance = sqrt(sum);
            // проверка, входит ли точка в окрестность
            if (distance <= d_epsilon) {
                neighbours[idx] = 1;
            } else {
                neighbours[idx] = 0;
            }
        } else {
            neighbours[idx] = 0;
        }
        // для подсчёта соседей
        if (neighbours[idx] == 1) {
            atomicAdd(count, 1);
        }
    }
}

// функция для расширения кластера
void expand_cluster(float* data, int start_point, int cluster_id, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // создание стека для отслеживания точек, которые нужно обработать
    stack<int> stack;

    // В стек добавляется начальная точка
    stack.push(start_point);

    // Пока стек не пустой
    while (!stack.empty()) {
        // получение точки из стека
        int point = stack.top();
        // удаление точки из стека
        stack.pop();

        // назначение точке метки текущего кластера
        labels[point] = cluster_id;

        // Запуск ядра для обнуление счётчика соседей
        nullCountCUDA<<<1, 1>>>(d_count);
        cudaDeviceSynchronize();

        // Поиск соседей
        findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point, d_neighbours, d_count);
        cudaDeviceSynchronize();

        int count;
        // копирование найденного количества соседей с GPU на CPU
        cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

        if (count >= minPts) {
            int* neighbours = new int[size_mass];
            cudaMemcpy(neighbours, d_neighbours, size_mass * sizeof(int), cudaMemcpyDeviceToHost);
            for (int i = 0; i < size_mass; i++) {
                if (neighbours[i] == 1) {
                    if (labels[i] == -1) {
                        stack.push(i);
                        labels[i] = cluster_id;
                    }
                    if (labels[i] == -2) {
                        labels[i] = cluster_id;
                    }
                }
            }
            delete[] neighbours;
        }
    }
}

// Основная фнункция алгоритма
int dbscan(float* data, int* labels, float* d_data, int* d_neighbours, int* d_count) {
    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_epsilon, &epsilon, sizeof(float));
    cudaMemcpyToSymbol(d_minPts, &minPts, sizeof(int));

    // Определение параметров для запуска ядер
    int blockSize = 256;
    int gridSize = (size_mass + blockSize - 1) / blockSize;

    // Начальное значение индекса кластера
    int cluster_id = 0;

    for (int point_index = 0; point_index < size_mass; point_index++) {
        // если точка не посещена
        if (labels[point_index] == -1) {
            // Запуск ядра для обнуление счётчика соседей
            nullCountCUDA<<<1, 1>>>(d_count);
            cudaDeviceSynchronize();

            // Поиск соседей
            findNeighboursCUDA<<<gridSize, blockSize>>>(d_data, point_index, d_neighbours, d_count);
            cudaDeviceSynchronize();

            int count;
            // копирование найденного количества соседей с GPU на CPU
            cudaMemcpy(&count, d_count, sizeof(int), cudaMemcpyDeviceToHost);

            // Если количество соседей меньше заданного минимума
            if (count < minPts) {
                // Точка помечается как шумовая (-2)
                labels[point_index] = -2;
                continue;
            }

            // Если больше или равно
            if (count >= minPts) {
                // Расширение кластера
                expand_cluster(data, point_index, cluster_id, labels, d_data, d_neighbours, d_count);
                // После того, как кластер завершён, увеличивается индекс для текущего
                cluster_id++;
            }
        }
    }

    return cluster_id;
}

int main() {


//  ***************** Начало замеров **********************
    for (int iter = 0; iter < iterations; iter++) {

    // Выделение памяти

    // Данные с точками
    float* myArray = new float[size_mass * dim];

    // для меток точек, изначально не посещённые (-1)
    int* labels = new int[size_mass];
    for (int i = 0; i < size_mass; i++) {
        labels[i] = -1;
    }

    // чтение данных их файла и запись в массив
    readFromFile(myArray, file);


        // -------------------------------------------------------------------------
        // замеры времени выделения памяти на GPU с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

    // Указатели на GPU память
    float* dev_data;
    int* dev_neighbours;
    int* d_count;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_neighbours, size_mass * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));


        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------

        // __________________
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);


    // Копирование данных на GPU
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);



        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // __________________



        // ...........................................................................
        // замеры времени работы алгоритма DBSCAN с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);

    // работа алгоритма DBSCAN, вернёт количество кластеров
    int count_clusters = dbscan(myArray, labels, dev_data, dev_neighbours, d_count);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float DBSCANTime;
        cudaEventElapsedTime(&DBSCANTime, start3, stop3);
        totalDBSCANTime += DBSCANTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................


        // .............................................................
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_neighbours);
    cudaFree(d_count);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ..................................................................

    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] labels;


}
// *********************** Конец замера времени ***************************



    cout << "\nDBSCAN. For mass 100000 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "DBSCAN: " << totalDBSCANTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalDBSCANTime + totalFreeTime) / iterations << " ms" << endl;


    return 0;
}



Overwriting DBSCANTIME_100000_GPU.cu


In [ ]:
!nvcc DBSCANTIME_100000_GPU.cu -o DBSCANTIME_100000_GPU
# компилляция

In [ ]:
# Запуск
!./DBSCANTIME_100000_GPU


DBSCAN. For mass 100000 points
GPU memory allocation time: 0.138656 ms
Time to copy data from the host to the device: 0.361248 ms
DBSCAN: 48340.1 ms
Free GPU: 0.197632 ms
Total: 48340.8 ms
